# CUR _ XGBoost 

In [20]:
from time import sleep
import numpy as np
import pandas as pd

import scipy
import sklearn 
from sklearn.metrics import accuracy_score, classification_report

from scipy.stats import skew, kurtosis

In [21]:
folder_path = "C:/Users/qhrrl/CodeExamples/3.Paper/0.Dataset"

In [22]:
pd_train = pd.read_csv(folder_path + '/cur_training_all.csv')
pd_val = pd.read_csv(folder_path + '/cur_validation_all.csv')

In [23]:
print(pd_train.shape)
print(pd_val.shape)

print(len(pd_train))
print(len(pd_val))

(305534, 48)
(38210, 48)
305534
38210


In [24]:
np_train = pd_train.to_numpy()
np_val = pd_val.to_numpy()

tr_labels = np_train[:,-1]
y_valid = np_val[:, -1]

In [25]:
pd_train = pd_train.drop(columns = ['Date','WATT','LABEL'])
pd_val = pd_val.drop(columns = ['Date','WATT','LABEL'])

In [26]:
train = pd_train.to_numpy()
x_valid = pd_val.to_numpy()

In [27]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train, tr_labels, test_size=0.2, random_state=42)

In [28]:
from xgboost import XGBClassifier

"""
# 반드시 튜닝해야할 파라미터는  min_child_weight / max_depth / gamma
"""

xgb = XGBClassifier(
    learning_rate=0.1, 
    n_estimators=500, 
    max_depth=5,
    min_child_weight=3, 
    gamma=0.2, 
    subsample=0.6, 
    colsample_bytree=1.0,
    objective='multi:softprob', 
    nthread=4, 
    scale_pos_weight=1, 
    seed=27)

In [ ]:
from sklearn.model_selection import GridSearchCV

# grid search를 하고 싶을 경우 아래 Code를 주석 해제

parameters = {
    "learning_rate" : [0.1, 0.01, 0.001, 0.0001],
   # "eta"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
    "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
      "min_child_weight" : [ 1, 3, 5, 7 ],
      "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4],
      "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
      }

grid = GridSearchCV(xgb,
                    parameters, n_jobs=4,
                    scoring="neg_log_loss",
                    cv=5)
grid.fit(
    x_train.squeeze(), 
    y_train, 
    eval_metric="auc", 
    eval_set=[(x_train, y_train), (x_valid, y_valid)], 
    verbose=True)

##여기까지 실행했는데 계속 500epoch가 반복되고 코드가 끝나지 않아서 뒤에는 실행을 못햇습니다.

In [ ]:
#XGboost평가

from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

y_pred = xgb.predict(x_test)
y_pred_proba = xgb.predict_proba(x_test)[:, 1]

print("\nAbout xgboost model")
print("Accuracy : %.4g" % accuracy_score(y_test, y_pred))
print("AUC Score (training set): %f" % roc_auc_score(y_test, y_pred_proba))
print("F1 Score (training set): %f" % f1_score(y_test, y_pred))

In [ ]:
draw_confusion_matrix(xgb, x_test, y_test, "xgboost")

In [ ]:
draw_roc(xgb, x_test, y_test, "xgboost")